In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [10]:
# Output File (CSV)
output_data_file = "/Users/stephanietran/Desktop/BOOTCAMP/python-api-challenge/WeatherPy/weather_data.csv"

weather_df = pd.read_csv(output_data_file)

weather_df.head().drop(columns=["Unnamed: 0"])

,City,Lats,Lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pointe-noire,-4.7761,11.8635,25.04,94,75,1.54,CG,1668918395
1,birao,10.2849,22.7882,18.22,27,72,3.11,CF,1668918396
2,graaff-reinet,-32.2522,24.5308,12.74,61,0,1.01,ZA,1668918396
3,atuona,-9.8000,-139.0333,25.31,70,42,4.85,PF,1668918397
4,tanta,30.7885,31.0019,15.92,83,6,3.12,EG,1668918397


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

In [12]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lats", "Lngs"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

filtered_weather_df.head().dropna()

,Unnamed: 0,City,Lats,Lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pointe-noire,-4.7761,11.8635,25.04,94,75,1.54,CG,1668918395
1,1,birao,10.2849,22.7882,18.22,27,72,3.11,CF,1668918396
2,2,graaff-reinet,-32.2522,24.5308,12.74,61,0,1.01,ZA,1668918396
3,3,atuona,-9.8000,-139.0333,25.31,70,42,4.85,PF,1668918397
4,4,tanta,30.7885,31.0019,15.92,83,6,3.12,EG,1668918397


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lats"]
    lng = row["Lngs"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: pointe-noire.
Closest hotel is Kactus Lodge.
------------
Retrieving Results for Index 1: birao.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: graaff-reinet.
Closest hotel is Rietjiesbos Self Catering.
------------
Retrieving Results for Index 3: atuona.
Closest hotel is Villa Enata.
------------
Retrieving Results for Index 4: tanta.
Closest hotel is Hesham & Fouad Pharma Group Pharmacies.
------------
Retrieving Results for Index 5: navalmoral de la mata.
Closest hotel is Hotel Moya S A.
------------
Retrieving Results for Index 6: yellowknife.
Closest hotel is The Explorer Hotel.
------------
Retrieving Results for Index 7: faanui.
Closest hotel is Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 8: albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: cape town.
Closest hotel is Radisson Blu Hotel Waterfront, Cape Town.
------------
Retrieving Results for I

In [17]:
hotel_df

,Unnamed: 0,City,Lats,Lngs,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,pointe-noire,-4.7761,11.8635,25.04,94,75,1.54,CG,1668918395,Kactus Lodge
1,1,birao,10.2849,22.7882,18.22,27,72,3.11,CF,1668918396,NaN
2,2,graaff-reinet,-32.2522,24.5308,12.74,61,0,1.01,ZA,1668918396,Rietjiesbos Self Catering
3,3,atuona,-9.8000,-139.0333,25.31,70,42,4.85,PF,1668918397,Villa Enata
4,4,tanta,30.7885,31.0019,15.92,83,6,3.12,EG,1668918397,Hesham & Fouad Pharma Group Pharmacies
...,...,...,...,...,...,...,...,...,...,...,...
566,566,oriximina,-1.7656,-55.8661,24.52,77,55,1.41,BR,1668918478,Pousada Tia Amélia
567,567,acatic,20.7833,-102.8833,14.46,76,83,0.90,MX,1668918779,Hacienda La Loma
568,568,florianopolis,-27.6146,-48.5012,18.68,80,20,3.60,BR,1668918779,Hotel Boutique Quinta das Videiras
569,569,poum,-20.2333,164.0167,25.41,78,73,5.64,NC,1668918780,NaN


In [20]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lats", "Lngs"]]
markers = gmaps.marker_layer(locations)

info_box_template = """

Hotel Name
{Hotel Name}

City
{City}
Country
{Country}

"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))